# 1:  Deep N-grams

In this we will explore Recurrent Neural Networks `RNN`.

In this notebook we will apply the following steps:
- Convert a line of text into a tensor
- Create a tensorflow dataset
- Define a GRU model using `TensorFlow`
- Train the model using `TensorFlow`
- Compute the accuracy of our model using the perplexity
- Generate text using our own model

## Table of Contents

- [Overview](#0)
- [1 - Data Preprocessing Overview](#1)
    - [1.1 - Loading in the Data](#1-1)
    - [1.2 - Create the vocabulary](#1-2)
    - [1.3 - Convert a Line to Tensor](#1-3)
        - [line_to_tensor](#ex-1)
    - [1.4 - Prepare our data for training and testing](#1-4)
    - [1.5 - Tensorflow dataset](#1-5)
    - [1.6 - Create the input and the output for our model](#1-6)
        - [create_batch_dataset](#ex-2)
    - [1.7 - Create the training dataset](#1-7)        
- [2 - Defining the GRU Language Model (GRULM)](#2)
    - [GRULM](#ex-3)
- [3 - Training](#3)
    - [compile_model](#ex-4)
- [4 - Evaluation](#4)
    - [4.1 - Evaluating using the Deep Nets](#4-1)
    - [log_perplexity](#ex-5)
- [5 - Generating Language with our Own Model](#5)
    - [GenerativeModel](#ex-6)
- [On statistical methods](#6)

<a name='0'></a>
## Overview

In this, you'll delve into the world of text generation using Recurrent Neural Networks (RNNs). our primary objective is to predict the next set of characters based on the preceding ones. This seemingly straightforward task holds immense practicality in applications like predictive text and creative writing.

The journey unfolds as follows:

- Data Preprocessing: You'll start by converting lines of text into numerical tensors, making them machine-readable.

- Dataset Creation: Next, you'll create a TensorFlow dataset, which will serve as the backbone for supplying data to our model.

- Neural Network Training: our model will be trained to predict the next set of characters, specifying the desired output length.

- Character Embeddings: Character embeddings will be employed to represent each character as a vector, a fundamental technique in natural language processing.

- GRU Model: our model utilizes a Gated Recurrent Unit (GRU) to process character embeddings and make sequential predictions. The following figure gives us a summary of what we are about to implement. 

<img src = "images/model.png" style="width:600px;height:150px;"/>

- Prediction Process: The model's predictions are achieved through a linear layer and log-softmax computation.

This overview sets the stage for our exploration of text generation. Get ready to unravel the secrets of language and embark on a journey into the realm of creative writing and predictive text generation.

And as usual let's start by importing all the required libraries.

In [1]:
import os
import traceback
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import shutil
import numpy as np
import random as  rnd

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input

from termcolor import colored

# set random seed
rnd.seed(32)

<a name='1'></a>
## 1 - Data Preprocessing Overview

<img src = "images/shakespeare.png" style="width:250px;height:250px;"/>

In this section, we will prepare the data for training our model. The data preparation involves the following steps:

- Dataset Import: Begin by importing the dataset. Each sentence is structured as one line in the dataset. To ensure consistency, remove any extra spaces from these lines using the `strip` function.

- Data Storage: Store each cleaned line in a list. This list will serve as the foundational dataset for our text generation task.

- Character-Level Processing: Since the goal is character generation, it's essential to process the text at the character level, not the word level. This involves converting each individual character into a numerical representation. To achieve this:

  - Use the [`tf.strings.unicode_split`](https://www.tensorflow.org/api_docs/python/tf/strings/unicode_split) function to split each sentence into its constituent characters.
  - Utilize [`tf.keras.layers.StringLookup`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/StringLookup) to map these characters to integer values. This transformation lays the foundation for character-based modeling.

- TensorFlow Dataset Creation: Create a TensorFlow dataset capable of producing data in batches. Each batch will consist of `batch_size` sentences, with each sentence containing a maximum of `max_length` characters. This organized dataset is essential for training our character generation model.

These preprocessing steps ensure that our dataset is meticulously prepared for the character-based text generation task, allowing us to work seamlessly with the Shakespearean corpus data.

<a name='1-1'></a>
### 1.1 - Loading in the Data

In [2]:
dirname = 'data/'
filename = 'shakespeare_data.txt'
lines = [] # storing all the lines in a variable. 

counter = 0

with open(os.path.join(dirname, filename)) as files:
    for line in files:        
        # remove leading and trailing whitespace
        pure_line = line.strip()#.lower()

        # if pure_line is not the empty string,
        if pure_line:
            # append it to the list
            lines.append(pure_line)
            
n_lines = len(lines)
print(f"Number of lines: {n_lines}")

Number of lines: 125097


In [3]:
print("\n".join(lines[506:514]))

BENVOLIO	Here were the servants of your adversary,
And yours, close fighting ere I did approach:
I drew to part them: in the instant came
The fiery Tybalt, with his sword prepared,
Which, as he breathed defiance to my ears,
He swung about his head and cut the winds,
Who nothing hurt withal hiss'd him in scorn:
While we were interchanging thrusts and blows,


<a name='1-2'></a>
### 1.2 - Create the vocabulary

In the following code cell, we will create the vocabulary for text processing. The vocabulary is a crucial component for understanding and processing text data. Here's what the code does:

- Concatenate all the lines in our dataset into a single continuous text, separated by line breaks.

- Identify and collect the unique characters that make up the text. This forms the basis of our vocabulary.

- To enhance the vocabulary, introduce two special characters:

  - [UNK]: This character represents any unknown or unrecognized characters in the text.
  - "" (empty character): This character is used for padding sequences when necessary.
- The code concludes with the display of statistics, showing the total count of unique characters in the vocabulary and providing a visual representation of the complete character set.racter set.

In [4]:
text = "\n".join(lines)
# The unique characters in the file
vocab = sorted(set(text))
vocab.insert(0,"[UNK]") # Add a special character for any unknown
vocab.insert(1,"") # Add the empty character for padding.

print(f'{len(vocab)} unique characters')
print(" ".join(vocab))

82 unique characters
[UNK]  	 
   ! $ & ' ( ) , - . 0 1 2 3 4 5 6 7 8 9 : ; ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z [ ] a b c d e f g h i j k l m n o p q r s t u v w x y z |


<a name='1-3'></a>
### 1.3 - Convert a Line to Tensor

Now that we have our list of lines, we will convert each character in that list to a number using the order given by our vocabulary. we can use [`tf.strings.unicode_split`](https://www.tensorflow.org/api_docs/python/tf/strings/unicode_split) to split the text into characters. 

In [5]:
line = "Hello world!"
chars = tf.strings.unicode_split(line, input_encoding='UTF-8')
print(chars)

tf.Tensor([b'H' b'e' b'l' b'l' b'o' b' ' b'w' b'o' b'r' b'l' b'd' b'!'], shape=(12,), dtype=string)


Using our vocabulary, we can convert the characters given by `unicode_split` into numbers. The number will be the index of the character in the given vocabulary. 

In [6]:
print(vocab.index('a'))
print(vocab.index('e'))
print(vocab.index('i'))
print(vocab.index('o'))
print(vocab.index('u'))
print(vocab.index(' '))
print(vocab.index('2'))
print(vocab.index('3'))

55
59
63
69
75
4
16
17


Tensorflow has a function [`tf.keras.layers.StringLookup`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/StringLookup)  that does this efficiently for list of characters. Note that the output object is of type `tf.Tensor`. Here is the result of applying the StringLookup function to the characters of "Hello world"

<a name='ex-1'></a>
### line_to_tensor

function that takes in a single line and transforms each character into its unicode integer.  This returns a list of integers, which we'll refer to as a tensor.

In [7]:
# FUNCTION: line_to_tensor
def line_to_tensor(line, vocab):
    """
    Converts a line of text into a tensor of integer values representing characters.

    Args:
        line (str): A single line of text.
        vocab (list): A list containing the vocabulary of unique characters.

    Returns:
        tf.Tensor(dtype=int64): A tensor containing integers (unicode values) corresponding to the characters in the `line`.
    """
    ###

    # Split the input line into individual characters
    chars = tf.strings.unicode_split(line, input_encoding='UTF-8')
    
    # Map characters to their respective integer values using StringLookup
    ids = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)(chars)
    
    ###

    return ids

In [8]:
# Test our function
tmp_ids = line_to_tensor('abc xyz', vocab)
print(f"Result: {tmp_ids}")
print(f"Output type: {type(tmp_ids)}")

Result: [55 56 57  4 78 79 80]
Output type: <class 'tensorflow.python.framework.ops.EagerTensor'>


we will also need a function that produces text given a numeric tensor. This function will be useful for inspection when we use our model to generate new text, because we will be able to see words rather than lists of numbers. The function will use the inverse Lookup function `tf.keras.layers.StringLookup` with `invert=True` in its parameters.

In [9]:
def text_from_ids(ids, vocab):
    """
    Converts a tensor of integer values into human-readable text.

    Args:
        ids (tf.Tensor): A tensor containing integer values (unicode IDs).
        vocab (list): A list containing the vocabulary of unique characters.

    Returns:
        str: A string containing the characters in human-readable format.
    """
    # Initialize the StringLookup layer to map integer IDs back to characters
    chars_from_ids = tf.keras.layers.StringLookup(vocabulary=vocab, invert=True, mask_token=None)
    
    # Use the layer to decode the tensor of IDs into human-readable text
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

Use the function for decoding the tensor produced by "Hello world!"

text_from_ids(ids, vocab).numpy()

<a name='1-4'></a>
### 1.4 - Prepare our data for training and testing
As usual, we will need some data for training our model, and some data for testing its performance. So, we will use 124097 lines for training and 1000 lines for testing. 

In [10]:
train_lines = lines[:-1000] # Leave the rest for training
eval_lines = lines[-1000:] # Create a holdout validation set

print(f"Number of training lines: {len(train_lines)}")
print(f"Number of validation lines: {len(eval_lines)}")

Number of training lines: 124097
Number of validation lines: 1000


<a name='1-5'></a>
### 1.5 - TensorFlow dataset

Most of the time in Natural Language Processing, and AI in general we use batches when training our models. Here, we will build a dataset that takes in some text and returns a batch of text fragments (Not necesarly full sentences) that we will use for training.
- The generator will produce text fragments encoded as numeric tensors of a desired length

Once we create the dataset, we can iterate on it like this:

```
data_generator.take(1)
```

This generator returns the data in a format that we could directly use in our model when computing the feed-forward of our algorithm. This batch dataset generator returns batches of data in an endless way. 

In order to get a dataset generator that produces batches of fragments from the corpus, we first need to convert the whole text into a single line, and then transform it into a single big tensor. This is only possible if our data fits completely into memory, but that is the case here.

In [11]:
all_ids = line_to_tensor("\n".join(["Hello world!", "Generative AI"]), vocab)
all_ids

<tf.Tensor: shape=(26,), dtype=int64, numpy=
array([34, 59, 66, 66, 69,  4, 77, 69, 72, 66, 58,  5,  3, 33, 59, 68, 59,
       72, 55, 74, 63, 76, 59,  4, 27, 35], dtype=int64)>

Create a dataset out of a tensor like input. This initial dataset will dispatch numbers in packages of a specified length. For example, we can use it for getting the 10 first encoded characters of our dataset. To make it easier to read, we can use the `text_from_ids` function.

In [12]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
print([text_from_ids([ids], vocab).numpy() for ids in ids_dataset.take(10)])

[b'H', b'e', b'l', b'l', b'o', b' ', b'w', b'o', b'r', b'l']


But we can configure this dataset to produce batches of the same size each time. We could use this functionality to produce text fragments of a desired size (`seq_length + 1`). We will explain later why we need an extra character into the sequence.

In [13]:
seq_length = 10
data_generator = ids_dataset.batch(seq_length + 1, drop_remainder=True)

we can verify that the data generator produces encoded fragments of text of the desired length. For example, let's ask the generator to produce 2 batches of data using the function `data_generator.take(2)`

In [14]:
for seq in data_generator.take(2):
    print(seq)

tf.Tensor([34 59 66 66 69  4 77 69 72 66 58], shape=(11,), dtype=int64)
tf.Tensor([ 5  3 33 59 68 59 72 55 74 63 76], shape=(11,), dtype=int64)


But as usual, it is easier to understand if we print it in human readable characters using the 'text_from_ids' function.

In [15]:
i = 1
for seq in data_generator.take(2):
    print(f"{i}. {text_from_ids(seq, vocab).numpy()}")
    i = i + 1

1. b'Hello world'
2. b'!\nGenerativ'


<a name='1-6'></a>
### 1.6 - Create the input and the output for our model

In this we have to predict the next character in a sequence. The following function creates 2 tensors, each with a length of `seq_length` out of the input sequence of lenght `seq_length + 1`. The first one contains the first `seq_length` elements and the second one contains the last `seq_length` elements. For example, if we split the sequence `['H', 'e', 'l', 'l', 'o']`, we will obtain the sequences `['H', 'e', 'l', 'l']` and `['e', 'l', 'l', 'o']`. 

In [16]:
def split_input_target(sequence):
    """
    Splits the input sequence into two sequences, where one is shifted by one position.

    Args:
        sequence (tf.Tensor or list): A list of characters or a tensor.

    Returns:
        tf.Tensor, tf.Tensor: Two tensors representing the input and output sequences for the model.
    """
    # Create the input sequence by excluding the last character
    input_text = sequence[:-1]
    # Create the target sequence by excluding the first character
    target_text = sequence[1:]

    return input_text, target_text

Look the result using the following sequence of characters

In [17]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

The first sequence will be the input and the second sequence will be the expected output

Now, put all this together into a function to create our batch dataset generator

<a name='ex-2'></a>
### create_batch_dataset
Lets create a batch dataset from the input text. Here are some things we will need. 

- Join all the input lines into a single string. When we have a big dataset, we would better use a flow from directory or any other kind of generator.
- Transform our input text into numeric tensors
- Create a TensorFlow DataSet from our numeric tensors: Just feed the numeric tensors into the function [`tf.data.Dataset.from_tensor_slices`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices)
- Make the dataset produce batches of data that will form a single sample each time. This is, make the dataset produce a sequence of `seq_length + 1`, rather than single numbers at each time. we can do it using the `batch` function of the already created dataset. we must specify the length of the produced sequences (`seq_length + 1`). So, the sequence length produced by the dataset will `seq_length + 1`. It must have that extra element since we will get the input and the output sequences out of the same element. `drop_remainder=True` will drop the sequences that do not have the required length. This could happen each time that the dataset reaches the end of the input sequence.
- Use the `split_input_target` to split each element produced by the dataset into the mentioned input and output sequences.The input will have the first `seq_length` elements, and the output will have the last `seq_length`. So, after this step, the dataset generator will produce batches of pairs (input, output) sequences.
- Create the final dataset, using `dataset_xy` as the starting point. we will configure this dataset to shuffle the data during the generation of the data with the specified BUFFER_SIZE. For performance reasons, we would like that tensorflow pre-process the data in parallel with training. That is called [`prefetching`](https://www.tensorflow.org/guide/data_performance#prefetching), and it will be configured for you.

In [18]:
#FUNCTION: create_batch_dataset
def create_batch_dataset(lines, vocab, seq_length=100, batch_size=64):
    """
    Creates a batch dataset from a list of text lines.

    Args:
        lines (list): A list of strings with the input data, one line per row.
        vocab (list): A list containing the vocabulary.
        seq_length (int): The desired length of each sample.
        batch_size (int): The batch size.

    Returns:
        tf.data.Dataset: A batch dataset generator.
    """
    # Buffer size to shuffle the dataset
    # (TF data is designed to work with possibly infinite sequences,
    # so it doesn't attempt to shuffle the entire sequence in memory. Instead,
    # it maintains a buffer in which it shuffles elements).
    BUFFER_SIZE = 10000
    
    # For simplicity, just join all lines into a single line
    single_line_data  = "\n".join(lines)

    ###
    
    # Convert the string into a tensor of integer values
    all_ids = line_to_tensor(single_line_data, vocab)
    
    # Create a TensorFlow Dataset from the tensor of integer values
    ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
    
    # Create a batch dataset
    data_generator = ids_dataset.batch(seq_length + 1, drop_remainder=True)
    
    # Map each sequence to input-output pairs
    dataset_xy = data_generator.map(split_input_target)
    
    # Assemble the final dataset with shuffling, batching, and prefetching
    dataset = (
        dataset_xy
        .shuffle(BUFFER_SIZE)
        .batch(batch_size, drop_remainder=True)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )            
                                     
    ###
    
    return dataset

In [19]:
# test our function
tf.random.set_seed(1)
dataset = create_batch_dataset(train_lines[1:100], vocab, seq_length=16, batch_size=2)

print("Prints the elements into a single batch. The batch contains 2 elements: ")

for input_example, target_example in dataset.take(1):
    print("\n\033[94mInput0\t:", text_from_ids(input_example[0], vocab).numpy())
    print("\n\033[93mTarget0\t:", text_from_ids(target_example[0], vocab).numpy())
    
    print("\n\n\033[94mInput1\t:", text_from_ids(input_example[1], vocab).numpy())
    print("\n\033[93mTarget1\t:", text_from_ids(target_example[1], vocab).numpy())

Prints the elements into a single batch. The batch contains 2 elements: 

Input0	: b'and sight distra'

Target0	: b'nd sight distrac'


Input1	: b'when in his fair'

Target1	: b'hen in his fair '


<a name='1-6'></a>
### 1.7 - Create the training dataset

Now, we can generate our training dataset using the functions defined above. This will produce pairs of input/output tensors each time the batch generator creates an entry.

In [20]:
# Batch size
BATCH_SIZE = 64
dataset = create_batch_dataset(train_lines, vocab, seq_length=100, batch_size=BATCH_SIZE)

<a name='2'></a>
## 2 - Defining the GRU Language Model (GRULM)

Now that we have the input and output tensors, we will go ahead and initialize our model. we will be implementing the `GRULM`, gated recurrent unit model. To implement this model, we will be using `TensorFlow`. Instead of implementing the `GRU` from scratch, we will use the necessary methods from a built-in package. we can use the following packages when constructing the model: 

- `tf.keras.layers.Embedding`: Initializes the embedding. In this case it is the size of the vocabulary by the dimension of the model. [docs](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding)  
    - `Embedding(vocab_size, embedding_dim)`.
    - `vocab_size` is the number of unique words in the given vocabulary.
    - `embedding_dim` is the number of elements in the word embedding (some choices for a word embedding size range from 150 to 300, for example).
___

- `tf.keras.layers.GRU`: `TensorFlow` GRU layer. [docs](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU)) Builds a traditional GRU of rnn_units with dense internal transformations. You can read the paper here: https://arxiv.org/abs/1412.3555
    - `units`: Number of recurrent units in the layer. It must be set to `rnn_units`
    - `return_sequences`: It specifies if the model returns a sequence of predictions. Set it to `True`
    - `return_state`: It specifies if the model must return the last internal state along with the prediction. Set it to `True` 
___

- `tf.keras.layers.Dense`: A dense layer. [docs](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense). we must set the following parameters:
    - `units`: Number of units in the layer. It must be set to `vocab_size`
    - `activation`: It must be set to `log_softmax` function as described in the next line.
___

- `tf.nn.log_softmax`: Log of the output probabilities. [docs](https://www.tensorflow.org/api_docs/python/tf/nn/log_softmax)
    - we don't need to set any parameters, just set the activation parameter as `activation=tf.nn.log_softmax`.
___

<a name='ex-3'></a>
### 3 - GRULM
Now let's Implement the `GRULM` class below. we should be using all the methods explained above.


In [21]:
#CLASS: GRULM
class GRULM(tf.keras.Model):
    """
    A GRU-based language model that maps from a tensor of tokens to activations over a vocabulary.

    Args:
        vocab_size (int, optional): Size of the vocabulary. Defaults to 256.
        embedding_dim (int, optional): Depth of embedding. Defaults to 256.
        rnn_units (int, optional): Number of units in the GRU cell. Defaults to 128.

    Returns:
        tf.keras.Model: A GRULM language model.
    """
    def __init__(self, vocab_size=256, embedding_dim=256, rnn_units=128):
        super().__init__(self)

        ###

        # Create an embedding layer to map token indices to embedding vectors
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        # Define a GRU (Gated Recurrent Unit) layer for sequence modeling
        self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
        # Apply a dense layer with log-softmax activation to predict next tokens
        self.dense = tf.keras.layers.Dense(vocab_size, activation=tf.nn.log_softmax)
        
        ###
    
    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        # Map input tokens to embedding vectors
        x = self.embedding(x, training=training)
        if states is None:
            # Get initial state from the GRU layer
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        # Predict the next tokens and apply log-softmax activation
        x = self.dense(x, training=training)
        if return_state:
            return x, states
        else:
            return x

Now, we can define a new GRULM model. we must set the `vocab_size` to 82; the size of the embedding `embedding_dim` to 256, and the number of units that will have recurrent neural network `rnn_units` to 512

In [22]:
# Length of the vocabulary in StringLookup Layer
vocab_size = 82

# The embedding dimension
embedding_dim = 256

# RNN layers
rnn_units = 512

model = GRULM(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units = rnn_units)

In [23]:
# testing our model

try:
    # Simulate inputs of length 100. This allows to compute the shape of all inputs and outputs of our network
    model.build(input_shape=(BATCH_SIZE, 100))
    model.call(Input(shape=(100)))
    model.summary() 
except:
    print("\033[91mError! \033[0mA problem occurred while building our model. This error can occur due to wrong initialization of the return_sequences parameter\n\n")
    traceback.print_exc()

Model: "grulm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 256)          20992     
                                                                 
 gru (GRU)                   [(None, 100, 512),        1182720   
                              (None, 512)]                       
                                                                 
 dense (Dense)               (None, 100, 82)           42066     
                                                                 
Total params: 1,245,778
Trainable params: 1,245,778
Non-trainable params: 0
_________________________________________________________________


Now, let's use the model for predicting the next character using the untrained model. At the begining the model will generate only gibberish.

In [24]:
for input_example_batch, target_example_batch in dataset.take(1):
    print("Input: ", input_example_batch[0].numpy()) # Lets use only the first sequence on the batch
    example_batch_predictions = model(tf.constant([input_example_batch[0].numpy()]))
    print("\n",example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Input:  [55  4 57 55 75 73 59 66 59 73 73  4 60 55 68 74 55 73 79 11  3 27 68 58
  4 57 62 63 66 58 63 73 62  4 59 72 72 69 72 11  4 74 62 55 74  4 74 62
 59 79  4 55 72 59  4 55 60 72 55 63 58 25  3 28 63 58 73  4 74 62 59 67
  4 66 59 55 76 59  4 71 75 55 65 63 68 61 11  4 56 63 58 73  4 74 62 59
 67  4 60 59]

 (1, 100, 82) # (batch_size, sequence_length, vocab_size)


The output size is (1, 100, 82). We predicted only on the first sequence generated by the batch generator. 100 is the number of predicted characters. It has exactly the same length as the input. And there are 82 values for each predicted character. Each of these 82 real values are related to the logarithm likelihood of each character to be the next one in the sequence. The bigger the value, the higher the likelihood. As the network is not trained yet, all those values must be very similar and random. Just check the values for the last prediction on the sequence.

In [25]:
example_batch_predictions[0][99].numpy()

array([-4.414955 , -4.4210873, -4.4031324, -4.4064994, -4.407753 ,
       -4.4177003, -4.398801 , -4.414058 , -4.401907 , -4.3995175,
       -4.3981104, -4.4146614, -4.4030724, -4.4065   , -4.4072433,
       -4.426503 , -4.4073563, -4.41889  , -4.4142785, -4.4125185,
       -4.408944 , -4.398369 , -4.3900857, -4.3986354, -4.4018908,
       -4.40355  , -4.398396 , -4.425095 , -4.4168234, -4.4045587,
       -4.414793 , -4.423625 , -4.4079046, -4.387917 , -4.4044228,
       -4.400095 , -4.3967414, -4.418302 , -4.409538 , -4.413951 ,
       -4.39556  , -4.381966 , -4.4066544, -4.425118 , -4.383355 ,
       -4.4090037, -4.4043145, -4.417834 , -4.408311 , -4.4066057,
       -4.401214 , -4.4049563, -4.417294 , -4.401831 , -4.413064 ,
       -4.3971887, -4.406681 , -4.416766 , -4.4177737, -4.4150267,
       -4.4211144, -4.3954144, -4.418568 , -4.409894 , -4.4002156,
       -4.380377 , -4.4126773, -4.400333 , -4.4100676, -4.413075 ,
       -4.394363 , -4.418049 , -4.394827 , -4.396073 , -4.3953

And the simplest way to choose the next character is by getting the index of the element with the highest likelihood. So, for instance, the prediction for the last characeter would be:

In [26]:
last_character = tf.math.argmax(example_batch_predictions[0][99])
print(last_character.numpy())

65


And the prediction for the whole sequence would be:

In [27]:
sampled_indices = tf.math.argmax(example_batch_predictions[0], axis=1)
print(sampled_indices.numpy())

[23 41  6 71 48 25 44  8 44  8 21 41 65 23 50 73 73 25 36 17 28 28 28 48
 72 45  6 28 28 27  1  1 27  6 41 73 73 75 43 54 54 54  6 71 73 41 73  6
  6 24 41 41 73 44 41 41 23 23 23 22 27 59 27 17 56 56 25 41 29  6  6 73
 41 12 44 41 77 44 41 41 48 73 58  1  1 35 54 54 41 28 13 25 41 29  6  6
 73 41 29 65]


Those 100 numbers represent 100 predicted characters. However, humans cannot read this. So, let's print the input and output sequences using our `text_from_ids` function, to check what is going on.

In [28]:
print("Input:\n", text_from_ids(input_example_batch[0], vocab))
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices, vocab))

Input:
 tf.Tensor(b'a causeless fantasy,\nAnd childish error, that they are afraid;\nBids them leave quaking, bids them fe', shape=(), dtype=string)

Next Char Predictions:
 tf.Tensor(b"9O$qV;R'R'7Ok9Xss;J3BBBVrS$BBAA$OssuQ]]]$qsOs$$:OOsROO9998AeA3bb;OC$$sO-ROwROOVsdI]]OB.;OC$$sOCk", shape=(), dtype=string)


As expected, the untrained model just produces random text as response of the given input. It is also important to note that getting the index of the maximum score is not always the best choice. In the last part of the notebook we will see another way to do it.

<a name='3'></a>
## 3 - Training

Now we are going to train our model. As usual, we have to define the cost function and the optimizer. we will use the following built-in functions provided by TensorFlow: 

- [`tf.losses.SparseCategoricalCrossentropy()`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy): The Sparce Categorical Cross Entropy loss. It is the loss function used for multiclass classification.    
    - `from_logits=True`: This parameter informs the loss function that the output values generated by the model are not normalized like a probability distribution. This is our case, since our GRULM model uses a `log_softmax` activation rather than the `softmax`.
- [`tf.keras.optimizers.Adam`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam): Use Adaptive Moment Estimation, a stochastic gradient descent method optimizer that works well in most of the cases. Set the `learning_rate` to 0.00125. 

<a name='ex-4'></a>
### compile_model

Let's Compile the GRULM model using a `SparseCategoricalCrossentropy` loss and the `Adam` optimizer

In [29]:
#FUNCTION: Compile model
def compile_model(model):
    """
    Sets the loss and optimizer for the given model

    Args:
        model (tf.keras.Model): The model to compile.

    Returns:
        tf.keras.Model: The compiled model.
    """
    ###

    # Define the loss function. Use SparseCategoricalCrossentropy
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
    # Define an Adam optimizer with the specified learning rate
    opt = tf.keras.optimizers.Adam(learning_rate=0.00125)
    
    # Compile the model using the parametrized Adam optimizer and the SparseCategoricalCrossentropy function
    model.compile(optimizer=opt, loss=loss)
    
    ###

    return model

Now, train our model for 30 epochs. With GPU this should take about one minute. With CPU this could take several minutes.

In [30]:
EPOCHS = 30

# Compile the model
model = compile_model(model)
# Fit the model
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/30
790/790 [==============================] - 18s 20ms/step - loss: 2.0157
Epoch 2/30
790/790 [==============================] - 17s 20ms/step - loss: 1.4810
Epoch 3/30
790/790 [==============================] - 17s 21ms/step - loss: 1.3801
Epoch 4/30
790/790 [==============================] - 18s 21ms/step - loss: 1.3336
Epoch 5/30
790/790 [==============================] - 18s 22ms/step - loss: 1.3044
Epoch 6/30
790/790 [==============================] - 19s 23ms/step - loss: 1.2829
Epoch 7/30
790/790 [==============================] - 19s 23ms/step - loss: 1.2659
Epoch 8/30
790/790 [==============================] - 19s 22ms/step - loss: 1.2527
Epoch 9/30
790/790 [==============================] - 19s 23ms/step - loss: 1.2415
Epoch 10/30
790/790 [==============================] - 19s 23ms/step - loss: 1.2320
Epoch 11/30
790/790 [==============================] - 19s 23ms/step - loss: 1.2240
Epoch 12/30
790/790 [==============================] - 19s 23ms/step - loss: 1.2169
E

In [31]:
import shutil
import os

output_dir = './model/'

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

os.makedirs(output_dir, exist_ok=True)  
model.save_weights(output_dir)

<a name='4'></a>
## 4 - Evaluation  
<a name='4-1'></a>
### 4.1 - Evaluating using the Deep Nets

Now that we have learned how to train a model, we will learn how to evaluate it. To evaluate language models, we usually use perplexity which is a measure of how well a probability model predicts a sample. Note that perplexity is defined as: 

$$P(W) = \sqrt[N]{\prod_{i=1}^{N} \frac{1}{P(w_i| w_1,...,w_{n-1})}}$$

As an implementation hack, we would usually take the log of that formula (to enable us to use the log probabilities we get as output of our `RNN`, convert exponents to products, and products into sums which makes computations less complicated and computationally more efficient). 


$$\log P(W) = {\log\left(\sqrt[N]{\prod_{i=1}^{N} \frac{1}{P(w_i| w_1,...,w_{n-1})}}\right)}$$$$ = \log\left(\left(\prod_{i=1}^{N} \frac{1}{P(w_i| w_1,...,w_{n-1})}\right)^{\frac{1}{N}}\right)$$
$$ = \log\left(\left({\prod_{i=1}^{N}{P(w_i| w_1,...,w_{n-1})}}\right)^{-\frac{1}{N}}\right)$$$$ = -\frac{1}{N}{\log\left({\prod_{i=1}^{N}{P(w_i| w_1,...,w_{n-1})}}\right)} $$$$ = -\frac{1}{N}{{\sum_{i=1}^{N}{\log P(w_i| w_1,...,w_{n-1})}}} $$

<a name='ex-5'></a>
### log_perplexity
Writing a program that will help evaluate our model. Implementation hack: our program takes in `preds` and `target`. `preds` is a tensor of log probabilities. we can use [`tf.one_hot`](https://www.tensorflow.org/api_docs/python/tf/one_hot) to transform the `target` into the same dimension. we then multiply them and sum them. For the sake of simplicity, we suggest you use the NumPy functions [`sum`](https://numpy.org/doc/stable/reference/generated/numpy.sum.html), [`mean`](https://numpy.org/doc/stable/reference/generated/numpy.mean.html) and [`equal`](https://numpy.org/doc/stable/reference/generated/numpy.equal.html), Good luck! 

In [32]:
# FUNCTION: log_perplexity
def log_perplexity(preds, target):
    """
    Function to calculate the log perplexity of a model.

    Args:
        preds (tf.Tensor): Predictions of a list of batches of tensors corresponding to lines of text.
        target (tf.Tensor): Actual list of batches of tensors corresponding to lines of text.

    Returns:
        float: The log perplexity of the model.
    """
    PADDING_ID = 1
    
    ###
    
    # Calculate log probabilities for predictions using one-hot encoding
    log_p = np.sum(preds * tf.one_hot(target, preds.shape[-1]), axis= -1) # HINT: tf.one_hot() should replace one of the Nones
    # Identify non-padding elements in the target
    non_pad = 1.0 - np.equal(target, PADDING_ID)          # we should check if the target equals to PADDING_ID
    # Apply non-padding mask to log probabilities to exclude padding
    log_p = log_p * non_pad                             # Get rid of the padding
    # Calculate the log perplexity by taking the sum of log probabilities and dividing by the sum of non-padding elements
    log_ppx = np.sum(log_p, axis = -1) / np.sum(non_pad, axis = -1) # Remember to set the axis properly when summing up
    # Compute the mean of log perplexity
    log_ppx = np.mean(log_ppx) # Compute the mean of the previous expression
        
    ###
    
    return -log_ppx

Now load the provided pretrained model just to ensure. we need to instantiate the GRULM model and then load the saved weights.

In [33]:
# Load the pretrained model. This step is optional. 
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 512

model = GRULM(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units = rnn_units)
model.build(input_shape=(100, vocab_size))
model.load_weights('./model/')


Now, we will use the 1000 lines of the corpus that were reserved at the begining of this notebook as test data. we will apply the same preprocessing as we did for the train dataset: get the numeric tensor from the input lines, and use the split_input_target to generate the inputs and the expected outputs.

Second, we will predict the next characters for the whole dataset, and we will compute the perplexity for the expected outputs and the given predictions.

In [34]:
#for line in eval_lines[1:3]:
eval_text = "\n".join(eval_lines)
eval_ids = line_to_tensor([eval_text], vocab)
input_ids, target_ids = split_input_target(tf.squeeze(eval_ids, axis=0))

preds, status = model(tf.expand_dims(input_ids, 0), training=False, states=None, return_state=True)

#Get the log perplexity
log_ppx = log_perplexity(preds, tf.expand_dims(target_ids, 0))
print(f'The log perplexity and perplexity of our model are {log_ppx} and {np.exp(log_ppx)} respectively')

The log perplexity and perplexity of our model are 1.2100799369743052 and 3.353752730680062 respectively


So, the log perplexity of the model is 1.21. It is not an easy to interpret metric, but it can be used to compare among models. The smaller the value the better the model.

<a name='5'></a>
## 5 - Generating Language with our Own Model

our GRULM model demonstrates an impressive ability to predict the most likely characters in a sequence, based on log scores. However, it's important to acknowledge that this model, in its default form, is deterministic and can result in repetitive and monotonous outputs. For instance, it tends to provide the same answer to a question consistently.

To make our language model more dynamic and versatile, we can introduce an element of randomness into its predictions. This ensures that even if we feed the model in the same way each time, it will generate different sequences of text.

To achieve this desired behavior, we can employ a technique known as random sampling. When presented with an array of log scores for the N characters in our dictionary, we add an array of random numbers to this data. The extent of randomness introduced into the predictions is regulated by a parameter called "temperature". By comparing the random numbers to the original input scores, the model adapts its choices, offering diversity in its outputs.

This doesn't imply that the model produces entirely random results on each iteration. Rather, with each prediction, there is a probability associated with choosing a character other than the one with the highest score. This concept becomes more tangible when we explore the accompanying Python code.

In [35]:
def temperature_random_sampling(log_probs, temperature=1.0):
    """Temperature Random sampling from a categorical distribution. The higher the temperature, the more 
       random the output. If temperature is close to 0, it means that the model will just return the index
       of the character with the highest input log_score
    
    Args:
        log_probs (tf.Tensor): The log scores for each characeter in the dictionary
        temperature (number): A value to weight the random noise. 
    Returns:
        int: The index of the selected character
    """
   # Generate uniform random numbers with a slight offset to avoid log(0)
    u = tf.random.uniform(minval=1e-6, maxval=1.0 - 1e-6, shape=log_probs.shape)
    
    # Apply the Gumbel distribution transformation for randomness
    g = -tf.math.log(-tf.math.log(u))
    
    # Adjust the logits with the temperature and choose the character with the highest score
    return tf.math.argmax(log_probs + g * temperature, axis=-1)

Now, it's time to bring all the elements together for the exciting task of generating new text. The GenerativeModel class plays a pivotal role in this process, offering two essential functions:

1. `generate_one_step`: This function is our go-to method for generating a single character at a time. It accepts two key inputs: an initial input sequence and a state that can be thought of as the ongoing context or memory of the model. The function delivers a single character prediction and an updated state, which can be used as the context for future predictions.

2. `generate_n_chars`: This function takes text generation to the next level. It orchestrates the iterative generation of a sequence of characters. At each iteration, generate_one_step is called with the last generated character and the most recent state. This dynamic approach ensures that the generated text evolves organically, building upon the context and characters produced in previous steps. Each character generated in this process is collected and stored in the result list, forming the final output text.

<a name='ex-6'></a>
### Generative Model
Implementing the One-Step Generator

In this, we will create a function to generate a single character based on the input text, using the provided vocabulary and the trained model. Follow these steps to complete the generate_one_step function:

1. Start by transforming our input text into a tensor using the given vocab. This will convert the text into a format that the model can understand.

2. Utilize the trained model with the input_ids and the provided states to predict the next characters. Make sure to retrieve the updated states from this prediction because they are essential for the final output.

3. Since we are only interested in the next character prediction, keep only the result for the last character in the sequence.

4. Employ the temperature random sampling technique to convert the vector of scores into a single character prediction. For this step, we will use the predicted_logits obtained in the previous step and the temperature parameter of the model.

5. To transform the numeric prediction into a human-readable character, use the text_from_ids function. Be mindful that text_from_ids expects a list as its input, so we need to wrap the output of the temperature_random_sampling function in square brackets [...]. Don't forget to use self.vocab as the second parameter for character mapping.

6. Finally, return the predicted_chars, which will be a single character, and the states tensor obtained from step 2. These components are essential for maintaining the sequence and generating subsequent characters.

In [36]:
# CLASS: GenerativeModel
class GenerativeModel(tf.keras.Model):
    def __init__(self, model, vocab, temperature=1.0):
        """
        A generative model for text generation.

        Args:
            model (tf.keras.Model): The underlying model for text generation.
            vocab (list): A list containing the vocabulary of unique characters.
            temperature (float, optional): A value to control the randomness of text generation. Defaults to 1.0.
        """
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.vocab = vocab
    
    @tf.function
    def generate_one_step(self, inputs, states=None):
        """
        Generate a single character and update the model state.

        Args:
            inputs (string): The input string to start with.
            states (tf.Tensor): The state tensor.

        Returns:
            tf.Tensor, states: The predicted character and the current GRU state.
        """
        # Convert strings to token IDs.
        
        ###

        # Transform the inputs into tensors
        input_ids = line_to_tensor(inputs, self.vocab)
        # Predict the sequence for the given input_ids. Use the states and return_state=True
        predicted_logits, states = self.model(input_ids, states=states, return_state = True)
        # Get only last element of the sequence
        predicted_logits = predicted_logits[0, -1, :]                      
        # Use the temperature_random_sampling to generate the next character. 
        predicted_ids = temperature_random_sampling(predicted_logits, self.temperature)
        # Use the chars_from_ids to transform the code into the correSponding char
        predicted_chars = text_from_ids([predicted_ids], self.vocab)
        
        ###
        
        # Return the characters and model state.
        return tf.expand_dims(predicted_chars, 0), states
    
    def generate_n_chars(self, num_chars, prefix):
        """
        Generate a text sequence of a specified length, starting with a given prefix.

        Args:
            num_chars (int): The length of the output sequence.
            prefix (string): The prefix of the sequence (also referred to as the seed).

        Returns:
            str: The generated text sequence.
        """
        states = None
        next_char = tf.constant([prefix])
        result = [next_char]
        for n in range(num_chars):
            next_char, states = self.generate_one_step(next_char, states=states)
            result.append(next_char)

        return tf.strings.join(result)[0].numpy().decode('utf-8')

In [37]:
tf.random.set_seed(272)
gen = GenerativeModel(model, vocab, temperature=0.5)

print(gen.generate_n_chars(32, " "), '\n\n' + '_'*80)
print(gen.generate_n_chars(32, "Dear"), '\n\n' + '_'*80)
print(gen.generate_n_chars(32, "KING"), '\n\n' + '_'*80)

 me to come again,
And when the f 

________________________________________________________________________________
Dear souls fall into the sea to see  

________________________________________________________________________________
KING OF FRANCE	Then therefore gentle 

________________________________________________________________________________


Now, generate a longer text. Let's check if it looks like Shakespeare fragment

In [38]:
tf.random.set_seed(np.random.randint(1, 1000))
gen = GenerativeModel(model, vocab, temperature=0.8)
import time
start = time.time()
print(gen.generate_n_chars(1000, "ROMEO "), '\n\n' + '_'*80)
print('\nRun time:', time.time() - start)

ROMEO the rest of Paris bears]
QUEEN MARGARET	Bound put the sound of Edward hath a son.
There is a little head a hundred knings; and
That thou wilt not defend me: when, my lord,
Then the man that I have done.
First Senator	Steel so forth this, when it is the kingdom,
That was worthy the dead to think upon you:
He is my ear. Take me to pardon me
From woe her lords and good ones, with a shine.
Then whoreson dead man with the behalf Pompey?
The prince of God,
This groan forlorn with her bottle hath
The finly shall we bear with fears with servants as
The city with the disportune doth not see
The greater chain will be bestowed of hows,
Our person by his service becomes the right.
YORK	He was full of tail heaven. You left him calls,
Your honour reigns to great mercy: I will betray
The crime of France, that lately I met upon:
They are sure that he was my wits at which never cannot
With the next night I have welcome.
CASSIUS	This is the hunter, stop the meaning of the wall.
The music cannot ca

In the generated text above, we can see that the model generates text that makes sense capturing dependencies between words and without any input. A simple n-gram model would have not been able to capture all of that in one sentence.

<a name='6'></a>
###  On statistical methods

Using a statistical method will not give you results that are as good as we saw here. our model will not be able to encode information seen previously in the data set and as a result, the perplexity will increase. Remember from Auto-Complete that the higher the perplexity, the worse our model is. Furthermore, statistical ngram models take up too much space and memory. As a result, they will be inefficient and too slow. Conversely, with deepnets, we can get a better perplexity. Note, learning about n-gram language models is still important and allows you to better understand deepnets.